In [1]:
import galprime as gp

from astropy.io import fits
from astropy.table import Table

import os

from tqdm import tqdm

In [8]:
def hdul_to_table(hdul):
    return Table.read(hdul, format='fits')


def combine_profile_sets(loc_1, loc_2, run_id_1, run_id_2, outdir):
    files_1 = [f.removeprefix(f'{run_id_1}') for f in os.listdir(loc_1) if f.endswith('.fits')]
    files_2 = [f.removeprefix(f'{run_id_2}') for f in os.listdir(loc_2) if f.endswith('.fits')]
    
    medians = {}

    for f in tqdm(files_1, desc=f"Combining profiles in {loc_1.split('/')[-2]}"):
        if f not in files_2:
           continue
        
        with fits.open(f"{loc_1}/{run_id_1}{f}") as hdul:
            data_1 = hdul[1:]

            with fits.open(f"{loc_2}/{run_id_2}{f}") as hdul:
                data_2 = hdul[1:]
        
                hdul_out_new = fits.HDUList()
                hdul_out_new.append(fits.PrimaryHDU())

                hdul_out_new.extend(data_1)
                hdul_out_new.extend(data_2)
                hdul_out_new.writeto(f"{outdir}/{f}", overwrite=True)

                profiles = [hdul_to_table(hdul) for hdul in data_1]
                profiles.extend([hdul_to_table(hdul) for hdul in data_2])

        smas, median, low, up = gp.bootstrap_median(profiles, dtype="table")

        medians[f] = [smas, median, low, up]

    return medians


def combine_outputs(filedir_1, filedir_2, run_id_1, run_id_2, outdir):

    output_files = gp.gen_filestructure(outdir)

    files_1 = gp.gen_filestructure(filedir_1, generate=False)
    files_2 = gp.gen_filestructure(filedir_2, generate=False)

    to_combine = ["MODEL_PROFS", "COADD_PROFS", "BGSUB_PROFS"]
    median_sets = []
    for key in to_combine:
        median_sets.append(combine_profile_sets(files_1[key], files_2[key], 
                                                run_id_1, run_id_2, 
                                                output_files[key]))
    bare_sets, coadd_sets, bgsub_sets = median_sets
    for f in bare_sets.keys():
        bare_table = gp.gen_median_table(*bare_sets[f])
        coadd_table = gp.gen_median_table(*coadd_sets[f])
        bgsub_table = gp.gen_median_table(*bgsub_sets[f])

        gp.gen_median_hdul(bare_table, coadd_table, bgsub_table, 
                           outname=f'{output_files["MEDIANS"]}{f}')


combine_outputs("gprime_sims/gprime_out/", "gprime_sims/gprime_out_1/", 
                12261545, 12252008,  "gprime_primary/")

Combining profiles in bgsub_profiles: 100%|██████████| 24/24 [06:31<00:00, 16.32s/it]
